In [ ]:
#-----------------------------------------------------------------------------------------------------
# 문서들을 embed하는 예제-3
# - 여기서는 문서들을 전처리하고, 임베딩 하는 과정임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    1.문서에서 문장들 나눔-문장들을 LEN 만큼 나눠서 임시 chunk 생성
#    - USE_GPT_PREPROCESSING=False : chunk 그대로 사용
#    - USE_GPT_PREPROCESSING=True : chunk를 GPT에 입력해서 chunk 내용 정리함.
#    2. 불용어 제거 -문장별루 분할.
#    3. 임베딩 설정값에 따라 ES에 chunk 임베딩
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
#-----------------------------------------------------------------------------------------------------
import openai  
import os
import sys
import random
import numpy as np
import pandas as pd
import time
import random
from tqdm import tqdm
from typing import Union, Dict, List, Optional

sys.path.append('..')
from utils import MyUtils
from utils import get_sentences, generate_text_GPT2, generate_text_davinci

myutils = MyUtils(yam_file_path='../data/settings.yaml')
settings = myutils.get_options()

myutils.seed_everything()  # seed 설정
DEVICE = myutils.GPU_info() # GPU 혹은 CPU

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

DATA_FOLDER = '../doc/company_2024/'  # text 추출된 파일이 있는 폴더
SPLIT_SENTENCE_LEN = 30 # chunk만들 문장 계수 = doc를 x계수만큼씩 문장으로 나눠서 chunk 만듬.

# gpt 정보 설정-----------------------------------------------------------------
USE_GPT_PREPROCESSING = True  # True= GPT로 전처리 수행 함, False=전처리 수행 안함.

# USE_GPT_PREPROCESSING=True일때 적용됨
GPT_MODEL:str = settings['GPT_MODEL']   # GPT 모델 종류 
MAX_TOKENS = 4096 #토큰 수  
TEMPERATURE = 1.0 # temperature 0~2 범위 : 작을수록 정형화된 답변, 클수록 유연한 답변(2는 엉뚱한 답변을 하므로, 1.5정도가 좋은것 같음=기본값은=1)
TOP_P = 0.2       #기본값은 1 (0.1이라고 하면 10% 토큰들에서 출력 토큰들을 선택한다는 의미)
STREAM = False    # 스트림으로 출력할지(실시간)

openai.api_key = settings['GPT_TOKEN']# **GPT  key 지정
#PROMPT = 'Q:위 내용들을 잘 정리해 주세요.'
PROMPT = 'Q:위내용에서 주요 주제를 파악하고 그에 따라 문장들을 구조화해주세요. 또한 문장에서 나온 사실적 정보들을 정리하여 제시해주세요'

#------------------------------------------------------------------------------------

# ES 임베딩 설정---------------------------------------------------------------------
OUT_DIMENSION = 128   # 128 혹은 768이면 0입력
EMBEDDING_METHOD=0   # 0=클러스터링 임베딩, 1=문장평균임베딩, 2=문장임베딩
NUM_CLUSTERS=10      # 클러스터링 임베딩일때 클러스터링 수 
NUM_CLUSTERS_VARIABLE=False # 클러스터링 임베딩일때 클러스터링을 문장계수마다 다르계할지.
CLUSTRING_MODE = "kmeans"  # "kmeans" = k-평균 군집 분석, kmedoids =  k-대표값 군집 분석
OUTMODE = "mean"           # kmeans 일때=>mean=평균임베딩값, max=최대임베딩값.,  kmedoids 일때 =>mean=평균임베딩값, medoid=대표임베딩값.

MODEL_PATH = '../model/kpf-sbert-128d-v1'#'../../../data11/model/kpf-sbert-v1.1'
POLLING_MODE = 'mean' # 폴링모드 

FLOAT_TYPE = 'float16' # float32 혹은 float16
SEED = 111

# ES 접속
ES_URL = 'http://192.168.0.51:9200/'             # es 접속 주소
ES_INDEX_NAME = 'qaindex_128_10_sentence_30_gpt_1'     # 생성혹은 추가할 인덱스 명
ES_INDEX_FILE = '../data/mpower10u_128d_10.json'  # 인덱스 구조 파일경로
BATCH_SIZE=20       # ES 배치 사이즈
CREATE_INDEX = True # True이면 기존에 인덱스가 있다면 제거하고 다시 생성.
#------------------------------------------------------------------------------------

In [ ]:
# DATA_FOLDER에 파일들을 불러와서 DF로 만듬.
# -문서는 MAX_LEN 길이만큼씩 잘라서 만듬.

# 파일이 여러개인 경우 폴더 지정
files = myutils.getListOfFiles(DATA_FOLDER)
assert len(files) > 0 # files가 0이면 assert 발생
print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

# 파일 경로를 지정하면됨.
#files = ["../data11/mpower_doc/신입사원교육.txt",]
docid = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.
count = 0

titles = []
contextids = []
contexts = []

for idx, file_path in enumerate(files):
    if '.ipynb_checkpoints' not in file_path:
        docid += 1
        count = 0
        
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            # 파일명만 추출하여 titles에 저장
            filename, ext = os.path.splitext(os.path.basename(file_path))  # 파일명과 확장자(.txt, .doc 등) 분리
            titles.append(filename)
            contextids.append(f'{docid}_{count}')
            contexts.append(data.strip()) # 두번째는 문서 제목+문서내용 합처서 문장만듬.
            
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, titles, contextids)), columns = ['context','question', 'contextid'])     

print(f'*doc 총 계수:{len(df_contexts)}\n')

In [ ]:
# JSON 파일로 저장해둠.
import json
examples = []

for context, title, contextid in zip(contexts, titles, contextids):
    doc = {}
    doc['context'] = context  # 문단
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("../doc/docs.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

In [ ]:
# 문장들로 분리
# => 이때는 chunk를 만들기 위한것이므로, remove_sentence_len=0, remove_duplication=False, remove_prefix_pattern= False로 지정함
doc_sentences = get_sentences(df=df_contexts, remove_sentence_len=2, remove_duplication=False, remove_prefix_pattern=False)

#print(f'len:{len(doc_sentences)}, 1: {doc_sentences[0]}')

In [ ]:
# GPT로 요약본 생성
def generate(settings:dict, chunk:str):

    input_prompt = chunk+'\n\n'+ PROMPT
    if GPT_MODEL.startswith("gpt-"):
        response, status = generate_text_GPT2(gpt_model=GPT_MODEL, prompt=input_prompt, system_prompt="", 
                                              assistants=[], stream=STREAM, timeout=20,
                                              max_tokens=MAX_TOKENS, temperature=TEMPERATURE, top_p=TOP_P) 
    else:
        response, status = generate_text_davinci(gpt_model=GPT_MODEL, prompt=input_prompt, stream=stream, timeout=20,
                                                 max_tokens=MAX_TOKENS, temperature=TEMPERATURE, top_p=TOP_P)

    return response, status

In [ ]:
# 분리된 문장들을 SPLIT_SENTENCE_LEN 계수만큼씩 분할해서 chunk 만듬
docs_split=[]
for count, doc in enumerate(doc_sentences):
    temp = []
    for i in range(0, len(doc), SPLIT_SENTENCE_LEN):
        chunk = doc[i:i+SPLIT_SENTENCE_LEN]
        if chunk:
            temp.append(chunk)
    if len(temp) > 0 and len(temp[-1]) < SPLIT_SENTENCE_LEN:
        remaining = temp.pop(-1)

        if temp:
            temp[-1].extend(remaining)
        else:
            temp.append(remaining)
    docs_split.append(temp)

#print(docs_split)

# X 씩 분할된 chunk에 title, contextid등을 붙임.
titles = []
contextids = []
contexts = []
org_contexts = []

for idx, docs in enumerate(tqdm(docs_split)):
    #print(idx)
    #paragraph_list = []
    count = 0
    for paragraph in docs:
        chunk_tmp = ''
        for sentence in paragraph:
            chunk_tmp +='\n'+sentence

        if chunk_tmp:
            title = df_contexts['question'][idx]
            titles.append(title)
            contextids.append(f'{idx}_{count}')
            org_contexts.append(title+'\n\n'+chunk_tmp.strip())    
            
            if USE_GPT_PREPROCESSING == True:
                # GPT에 입력해서 chunk_tmp 입력해서 정리함.
                chunk = title+'\n\n'+chunk_tmp.strip()
                response, status = generate(settings=settings, chunk=chunk)
                if count < 3:  # 문서당 3개만 출력함
                    print(f'\nin(status):{status}:\n{chunk}\n')
                    print(f'\nout:\n{response}')

                if status == 0:
                    contexts.append(title+'\n\n'+response.strip()) 
                else:
                    print(f'\ngpt error:\n{response}')
                    contexts.append(title+'\n\n'+chunk_tmp.strip()) #에러나면 원본을  context에 담음
            else:
                #paragraph_list.append(tmp[1:]) # 맨앞 공백은 제거
                contexts.append(title+'\n\n'+chunk_tmp.strip())    
                
            count +=1

# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, org_contexts, titles, contextids)), columns = ['context', 'org_context', 'question', 'contextid'])     
print(f'\r\n*chunk 총 계수:{len(df_contexts)}\n')

# JSON 파일로 저장해둠.(옵션)
import json
examples = []

for org_contexts, context, title, contextid in zip(org_contexts, contexts, titles, contextids):
    doc = {}
    doc['org_context'] = org_contexts        # 문단 원본 내용
    doc['context'] = context  # 문단내용(*GPT 요약인 경우 문단 org 내용과 다를수 있음.)
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
with open("../doc/chunk.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)
    

In [ ]:

import pandas as pd
import json

# 저장된 chunk.json 파일을 불러옴.
# JSON 파일을 읽어옵니다.
with open("../doc/chunk.json", "r") as f:
    data = json.load(f)

# 데이터 프레임으로 변환합니다.
df_contexts = pd.DataFrame(data['text'])

# 데이터프레임을 출력합니다.
#print(df_contexts)

In [ ]:
df_contexts['context'][2]

In [ ]:
# 임베딩 하기 위해 chunk를 문장들로 분리
# => 이때는 remove_sentence_len=3로 해서 3문자 이하는 제거 함.
doc_sentences = get_sentences(df=df_contexts, remove_sentence_len=3, remove_duplication=False)

#print(f'len:{len(doc_sentences)}, 1: {doc_sentences[1]}')

In [ ]:
# 인덱스 추가
from tqdm.notebook import tqdm
from utils import embed_text, bi_encoder, mpower_index_batch, create_index, clustering_embedding, kmedoids_clustering_embedding

# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
    
# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#---------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#---------------------------------------------------------------------------
def index_data(es, df_contexts, doc_sentences:list):
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_contexts['contextid'].values.tolist()
    rfile_texts = df_contexts['context'].values.tolist()

    if OUT_DIMENSION == 0:
        dimension = 768
    else:
        dimension = 128

    clustering_num = NUM_CLUSTERS
        
    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences-------------------')
            if len(sentences) > 5:
                print(sentences[:5])
            else:
                print(sentences)
                
            myutils.log_message(f'*[index_data] embeddings.shape: {embeddings.shape}', log_folder='../log/documet')
            print()
        
        #----------------------------------------------------------------
        multiple = 1
        
        # [bong][2023-04-28] 임베딩 출력 계수에 따라 클러스터링 계수를 달리함.
        if NUM_CLUSTERS_VARIABLE == True:
            embeddings_len = embeddings.shape[0]
            if embeddings_len > 2000:
                multiple = 6
            elif embeddings_len > 1000:
                multiple = 5 # 5배
            elif embeddings_len > 600:
                multiple = 4 # 4배
            elif embeddings_len > 300:
                multiple = 3 # 3배
            elif embeddings_len > 100:
                multiple = 2 # 2배
        #----------------------------------------------------------------
        
        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                # [bong][2023-04-28] 문장이 많은 경우에는 클러스터링 계수를 2,3배수로 함
                emb = clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
            clustering_num = 1  # 평균값일때는 NUM_CLUSTERS=1로 해줌.
        # 2=문장임베딩
        else:
            emb = embeddings

        if i < 3:
            myutils.log_message(f'*[index_data] cluster emb.shape: {emb.shape}', log_folder='../log/documet')
            print()
        
        #--------------------------------------------------- 
        # docs에 저장 
        #  [bong][2023-04-28] 여러개 벡터인 경우에는 벡터를 10개씩 분리해서 여러개 docs를 만듬.
        for j in range(multiple):
            count += 1
            doc = {}                                #dict 선언
            doc['rfile_name'] = rfile_names[i]      # contextid 담음
            doc['rfile_text'] = rfile_texts[i]      # text 담음.
            doc['dense_vectors'] = emb[j * clustering_num : (j+1) * clustering_num] # emb 담음.
            docs.append(doc)
        #---------------------------------------------------    

            if count % BATCH_SIZE == 0:
                mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
                docs = []
                myutils.log_message("[index_data](1) Indexed {} documents.".format(count), log_folder='../log/documet')
                 

    if docs:
        mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
        myutils.log_message("[index_data](2) Indexed {} documents.".format(count), log_folder='../log/documet')

    es.indices.refresh(index=ES_INDEX_NAME)

    myutils.log_message(f'*인덱싱 시간 : {time.time()-start:.4f}\n', log_folder='../log/documet')
    print()
#---------------------------------------------------------------------------


In [ ]:
# ES 접속해서 임베딩 처리함.
es = Elasticsearch(ES_URL)
create_index(es, ES_INDEX_FILE, ES_INDEX_NAME, create=CREATE_INDEX)

# 임베딩 모델 로딩
WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)
print()
try:
    index_data(es, df_contexts, doc_sentences)
except Exception as e:
    error = f'index_data fail'
    msg = f'{error}=>{e}'
    myutils.log_message(f'/embed/es {msg}', log_folder='../log/documet')